In [4]:
import sys
import pybind11

sys.path.append("./build/")   # add build/ to Python module search path
import simulator
import numpy as np
import matplotlib.pyplot as plt
import time

ImportError: dynamic module does not define module export function (PyInit_simulator)

In [5]:
print(sys.path)

['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.12/dist-packages', 'build', '.\\build', './build/']


In [ ]:
# Setup script for a Python package using pybind11
from setuptools import setup, Extension

ext_modules = [
    Extension(
        "example",
        ["example.cpp"],
        include_dirs=[pybind11.get_include()],
        language="c++"
    ),
]

setup(
    name="example",
    version="0.1",
    ext_modules=ext_modules,
)


In [2]:
import simulator

ModuleNotFoundError: No module named 'simulator'